In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc('font', family=font_name)

import folium
import json

# 관악구/강남구 동명 '신사동' -> 신사동(관) / 신사동(강)
geo_str = json.load(open('./데이터수집/skorea_submunicipalities_geo_simple.json', encoding='utf-8'))

In [2]:
# JSON 행정동 수
# len(pd.DataFrame(list(pd.DataFrame(geo_str['features'])['properties']))['name'].unique())

In [3]:
df_total = pd.read_csv("./전처리데이터/상권-추정매출_전처리.csv", index_col=0)
df_total

C:\Users\82108\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,기준_년_코드,기준_분기_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,남성_매출_비율,여성_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,연령대_50_매출_금액,연령대_60_이상_매출_금액,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,시군구명,읍면동명
0,2013,1,골목상권,1000001,계동길,한식음식점,698475768,15506,57,43,16,6,98488297,34939600,1800,799,0,종로구,가회동
1,2013,1,골목상권,1000001,계동길,중식음식점,125443486,6766,61,39,8,3,7523988,2779164,450,159,0,종로구,가회동
2,2013,1,골목상권,1000001,계동길,일식음식점,48533937,1769,47,53,7,3,2373187,1048677,96,35,0,종로구,가회동
3,2013,1,골목상권,1000001,계동길,양식음식점,139237013,3108,48,52,9,4,13059652,4331922,197,86,0,종로구,가회동
4,2013,1,골목상권,1000001,계동길,분식전문점,161453046,8565,40,60,5,2,7808953,1847629,419,114,0,종로구,가회동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093404,2020,1,골목상권,1000001,계동길,제과점,257814129,27092,41,59,17,12,42742868,31876953,4351,2989,4,종로구,가회동
1093405,2020,1,골목상권,1000001,계동길,양식음식점,156728306,3589,31,69,7,4,20924446,6812377,325,157,3,종로구,가회동
1093406,2020,1,골목상권,1000001,계동길,일식음식점,55484866,1272,55,45,28,14,14016411,5464850,314,162,3,종로구,가회동
1093407,2020,1,골목상권,1000001,계동길,중식음식점,25767161,1413,47,53,7,6,1514728,1526990,96,69,1,종로구,가회동


In [4]:
# JSON과 읍면동명 일치하게 수정
# '사당제1동' -> '사당1동'
# 관악구/강남구 동명 '신사동' -> 신사동(관) / 신사동(강)

for n in range(0,9):
    df_total["읍면동명"] = df_total["읍면동명"].str.replace(pat=r'제'+str(n), repl=str(n), regex=True)
df_total["읍면동명"] = df_total["읍면동명"].str.replace(pat='.', repl='·', regex=True)

df_total.at[(df_total["시군구명"]=="관악구")&(df_total["읍면동명"]=="신사동"), "읍면동명"] = "신사동(관)"
df_total.at[(df_total["시군구명"]=="강남구")&(df_total["읍면동명"]=="신사동"), "읍면동명"] = "신사동(강)"

In [5]:
# 2019년 이전과 이후 '서비스_업종_코드_명'이 상이하여 통일

df_total.loc[df_total['서비스_업종_코드_명']=='호프-간이주점', '서비스_업종_코드_명'] = '호프·간이주점'
df_total.loc[df_total['서비스_업종_코드_명']=='커피-음료', '서비스_업종_코드_명'] = '커피·음료'

df_total.loc[df_total['서비스_업종_코드_명']=='가구', '서비스_업종_코드_명'] = '가구·가전'
df_total.loc[df_total['서비스_업종_코드_명']=='가전제품', '서비스_업종_코드_명'] = '가구·가전'

df_total.loc[df_total['서비스_업종_코드_명']=='화초', '서비스_업종_코드_명'] = '화초·애완'
df_total.loc[df_total['서비스_업종_코드_명']=='애완동물', '서비스_업종_코드_명'] = '화초·애완'
df_total.loc[df_total['서비스_업종_코드_명']=='동물병원', '서비스_업종_코드_명'] = '화초·애완'

df_total.loc[df_total['서비스_업종_코드_명']=='자동차수리', '서비스_업종_코드_명'] = '자동차수리·세차'
df_total.loc[df_total['서비스_업종_코드_명']=='자동차미용', '서비스_업종_코드_명'] = '자동차수리·세차'

df_total.loc[df_total['서비스_업종_코드_명']=='미곡판매', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='주류도매', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='육류판매', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='반찬가게', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='청과상', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='수산물판매', '서비스_업종_코드_명'] = '식료품'
df_total.loc[df_total['서비스_업종_코드_명']=='건강보조식품', '서비스_업종_코드_명'] = '식료품'

df_total.loc[df_total['서비스_업종_코드_명']=='골프연습장', '서비스_업종_코드_명'] = '오락·운동'
df_total.loc[df_total['서비스_업종_코드_명']=='볼링장', '서비스_업종_코드_명'] = '오락·운동'
df_total.loc[df_total['서비스_업종_코드_명']=='운동/경기용품', '서비스_업종_코드_명'] = '오락·운동'

df_total.loc[df_total['서비스_업종_코드_명']=='예술학원', '서비스_업종_코드_명'] = '예체능학원'
df_total.loc[df_total['서비스_업종_코드_명']=='스포츠 강습', '서비스_업종_코드_명'] = '예체능학원'
df_total.loc[df_total['서비스_업종_코드_명']=='컴퓨터학원', '서비스_업종_코드_명'] = '예체능학원'

df_total.loc[df_total['서비스_업종_코드_명']=='서적', '서비스_업종_코드_명'] = '서적·문구'
df_total.loc[df_total['서비스_업종_코드_명']=='문구', '서비스_업종_코드_명'] = '서적·문구'
df_total.loc[df_total['서비스_업종_코드_명']=='완구', '서비스_업종_코드_명'] = '서적·문구'

df_total.loc[df_total['서비스_업종_코드_명']=='고시원', '서비스_업종_코드_명'] = '숙박업'
df_total.loc[df_total['서비스_업종_코드_명']=='여관', '서비스_업종_코드_명'] = '숙박업'

df_total.loc[df_total['서비스_업종_코드_명']=='의약품', '서비스_업종_코드_명'] = '의약·의료용품'
df_total.loc[df_total['서비스_업종_코드_명']=='의료기기', '서비스_업종_코드_명'] = '의약·의료용품'
df_total.loc[df_total['서비스_업종_코드_명']=='안경', '서비스_업종_코드_명'] = '의약·의료용품'

df_total.loc[df_total['서비스_업종_코드_명']=='미용실', '서비스_업종_코드_명'] = '두발미용업'

df_total.loc[df_total['서비스_업종_코드_명']=='컴퓨터및주변장치판매', '서비스_업종_코드_명'] = '컴퓨터·주변기기'

df_total.loc[df_total['서비스_업종_코드_명']=='유아의류', '서비스_업종_코드_명'] = '의류점'
df_total.loc[df_total['서비스_업종_코드_명']=='일반의류', '서비스_업종_코드_명'] = '의류점'
df_total.loc[df_total['서비스_업종_코드_명']=='한복점', '서비스_업종_코드_명'] = '의류점'

df_total.loc[df_total['서비스_업종_코드_명']=='전자상거래업', '서비스_업종_코드_명'] = '통신판매업'

df_total.loc[df_total['서비스_업종_코드_명']=='시계및귀금속', '서비스_업종_코드_명'] = '패션용품'
df_total.loc[df_total['서비스_업종_코드_명']=='가방', '서비스_업종_코드_명'] = '패션용품'
df_total.loc[df_total['서비스_업종_코드_명']=='신발', '서비스_업종_코드_명'] = '패션용품'

df_total.loc[df_total['서비스_업종_코드_명']=='세탁소', '서비스_업종_코드_명'] = '세탁소(가정)'

# 통일 불가한 '서비스_업종_코드_명'은 삭제 (단축 분류하기 때문에 영향 적음 - 확인)

df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="주방·가정용품"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="철물점"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="변호사사무소"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="기타법무서비스"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="세무사사무소"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="변리사사무소"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="자전거 및 기타운송장비"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="조명용품"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="인테리어"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="법무사사무소"].index)
df_total = df_total.drop(index=df_total[df_total["서비스_업종_코드_명"]=="회계사사무소"].index)

## column 단축을 위한 분류

In [6]:
# 서비스업 분류

service_dict = {
    'health' : ['한의원', '의약·의료용품', '치과의원', '일반의원', '피부관리실', '의료기기', '의약품', '안경'],
    'clothes' : ['의류점', '패션용품', '신발', '일반의류', '유아의류', '한복점'],
    'food' : ['한식음식점', '중식음식점', '일식음식점', '양식음식점', '분식전문점', '패스트푸드점', '치킨전문점', '커피·음료',
            '호프·간이주점', '커피-음료', '호프-간이주점', '식료품', '제과점', '반찬가게', '청과상',
            '수산물판매', '육류판매', '미곡판매', '주류도매', '슈퍼마켓', '편의점', '건강보조식품'],
    'etc' : ['화장품', '섬유제품', '주방·가정용품',
            '서적·문구', '가구·가전','통신판매업', '화초·애완', '핸드폰', '컴퓨터·주변기기',
           '인테리어', '애완동물', '화초', '시계및귀금속', '철물점', '문구', '조명용품',
           '전자상거래업', '가방', '가구', '서적', '운동/경기용품', '가전제품', '컴퓨터및주변장치판매', '완구',
           '자전거 및 기타운송장비', '일반교습학원', '예체능학원', '외국어학원', '스포츠 강습',
            '예술학원', '컴퓨터학원', '노래방', '당구장', '스포츠클럽', '오락·운동', 'PC방', '골프연습장', '볼링장', '네일숍',
            '숙박업', '여관', '고시원', '자동차수리·세차', '두발미용업', '세탁소(가정)', '미용실', '세탁소',
            '자동차수리', '자동차미용', '동물병원', '가전제품수리', '부동산중개업', '세무사사무소',
            '기타법무서비스', '변호사사무소', '회계사사무소', '법무사사무소', '변리사사무소']
}

In [7]:
# 서비스업 분류 column 추가

for key in service_dict.keys():
    df_total.loc[df_total["서비스_업종_코드_명"].isin(service_dict[key]), '서비스_업종_분류'] = key

In [8]:
df_total.loc[df_total["서비스_업종_코드_명"].isin(service_dict['food'])]

,기준_년_코드,기준_분기_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,남성_매출_비율,여성_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,연령대_50_매출_금액,연령대_60_이상_매출_금액,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,시군구명,읍면동명,서비스_업종_분류
0,2013,1,골목상권,1000001,계동길,한식음식점,698475768,15506,57,43,16,6,98488297,34939600,1800,799,0,종로구,가회동,food
1,2013,1,골목상권,1000001,계동길,중식음식점,125443486,6766,61,39,8,3,7523988,2779164,450,159,0,종로구,가회동,food
2,2013,1,골목상권,1000001,계동길,일식음식점,48533937,1769,47,53,7,3,2373187,1048677,96,35,0,종로구,가회동,food
3,2013,1,골목상권,1000001,계동길,양식음식점,139237013,3108,48,52,9,4,13059652,4331922,197,86,0,종로구,가회동,food
4,2013,1,골목상권,1000001,계동길,분식전문점,161453046,8565,40,60,5,2,7808953,1847629,419,114,0,종로구,가회동,food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093404,2020,1,골목상권,1000001,계동길,제과점,257814129,27092,41,59,17,12,42742868,31876953,4351,2989,4,종로구,가회동,food
1093405,2020,1,골목상권,1000001,계동길,양식음식점,156728306,3589,31,69,7,4,20924446,6812377,325,157,3,종로구,가회동,food
1093406,2020,1,골목상권,1000001,계동길,일식음식점,55484866,1272,55,45,28,14,14016411,5464850,314,162,3,종로구,가회동,food
1093407,2020,1,골목상권,1000001,계동길,중식음식점,25767161,1413,47,53,7,6,1514728,1526990,96,69,1,종로구,가회동,food


### 클러스터링 데이터 전처리 파일 출력

In [9]:
df_summary = pd.DataFrame(df_total[df_total["기준_년_코드"]==2019].groupby(["시군구명", "읍면동명", "서비스_업종_분류"]).size())
df_summary = df_summary.unstack()[0]
df_summary = df_summary.reset_index()
df_summary

서비스_업종_분류,시군구명,읍면동명,clothes,etc,food,health
0,강남구,개포2동,19.0,163.0,175.0,51.0
1,강남구,개포4동,37.0,218.0,276.0,70.0
2,강남구,논현1동,147.0,573.0,531.0,189.0
3,강남구,논현2동,155.0,611.0,674.0,191.0
4,강남구,대치1동,31.0,215.0,208.0,77.0
...,...,...,...,...,...,...
369,중랑구,상봉1동,3.0,37.0,44.0,6.0
370,중랑구,상봉2동,40.0,157.0,166.0,53.0
371,중랑구,신내1동,14.0,76.0,61.0,24.0
372,중랑구,중화1동,23.0,122.0,117.0,28.0


In [10]:
df_summary.to_csv('shop_summary(utf8).csv', encoding='utf-8-sig', index=False)